# TPOT Classifier
TPOT is a Tree-based Pipeline Optimization Tool for classification
Population Size: This parameter determines the number of different machine learning pipelines that are initially generated and evaluated in each generation of the evolutionary process.
Generations: This parameter controls the number of iterations or "generations" that the evolutionary process runs for.
New pipelines are created through genetic operations like mutation (slightly modifying existing pipelines) and crossover (combining parts of two existing pipelines).
Summary:
Population Size controls the breadth of the initial search.
Generations control the depth of the search, allowing TPOT to refine its solutions over time.

In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import torch.nn as nn
from tpot import TPOTClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

In [53]:
deals= pd.read_csv('Sample_Data_Deals2.csv')
#Change column type from Object to Category for columns 3,4,5
for col in ['Country', 'Industry', 'Deal Status']:
    deals[col] = deals[col].astype('category')

In [54]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   OrderID        100 non-null    object  
 1   OrderQuantity  100 non-null    int64   
 2   OrderValue     100 non-null    int64   
 3   Country        100 non-null    category
 4   Industry       100 non-null    category
 5   Deal Status    100 non-null    category
dtypes: category(3), int64(2), object(1)
memory usage: 3.4+ KB


In [55]:
#Drop Order ID
deals1 = pd.DataFrame(deals.iloc[:,1:])
deals1.head()

,OrderQuantity,OrderValue,Country,Industry,Deal Status
0,371,383,Canada,Technology,Won
1,163,121,Canada,Finance,Won
2,191,117,Australia,Manufacturing,Lost
3,150,143,Australia,Manufacturing,Lost
4,165,148,Australia,Manufacturing,Lost


In [56]:
deals1['Country'].nunique

<bound method IndexOpsMixin.nunique of 0        Canada
1        Canada
2     Australia
3     Australia
4     Australia
        ...    
95      Germany
96      Germany
97      Germany
98      Germany
99      Germany
Name: Country, Length: 100, dtype: category
Categories (5, object): ['Australia', 'Canada', 'China', 'France', 'Germany']>

In [57]:
deals1['Industry'].nunique

<bound method IndexOpsMixin.nunique of 0        Technology
1           Finance
2     Manufacturing
3     Manufacturing
4     Manufacturing
          ...      
95           Energy
96       Government
97       Government
98           Energy
99       Technology
Name: Industry, Length: 100, dtype: category
Categories (7, object): ['Energy', 'Finance', 'Government', 'Healthcare', 'Manufacturing', 'Retail', 'Technology']>

In [58]:
#Encoding categorical data, 
Country = {'Australia': 1, 'Canada':2, 'China':3, 'France':4, 'Germany':5}
deals1['Country'] = deals1['Country'].map(Country)

Industry = {'Energy':1, 'Finance':2, 'Government':3, 'Healthcare':4, 'Manufacturing':5, 'Retail':6, 'Technology':7}
deals1['Industry'] = deals1['Industry'].map(Industry)

dealstat= {'Won':1, 'Lost':0}
deals1['Deal Status'] = deals1['Deal Status'].map(dealstat)

print(deals1.head())

   OrderQuantity  OrderValue Country Industry Deal Status
0            371         383       2        7           1
1            163         121       2        2           1
2            191         117       1        5           0
3            150         143       1        5           0
4            165         148       1        5           0


In [59]:
#Create x and y datasets
X = deals1.drop('Deal Status',axis=1).values
y = deals1['Deal Status'].values

In [60]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [61]:
#Transform your data
scaler=MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [62]:
#Define Grid Search Paramters
param_grid = {
    'generations': [10, 20, 30],
    'population_size': [50, 100, 200],
}

# Define TPOT Classifier model & do Gridserach
tpot = TPOTClassifier(verbosity=2, 
    max_time_mins=3, 
    max_eval_time_mins=0.04,
    random_state=101) #To ensure reproducibility
grid_search = GridSearchCV(estimator=tpot, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6599999999999999

Generation 2 - Current best internal CV score: 0.6599999999999999

Generation 3 - Current best internal CV score: 0.6599999999999999

Generation 4 - Current best internal CV score: 0.6599999999999999

Generation 5 - Current best internal CV score: 0.6599999999999999

Generation 6 - Current best internal CV score: 0.6599999999999999

Generation 7 - Current best internal CV score: 0.6599999999999999

Generation 8 - Current best internal CV score: 0.6599999999999999

Generation 9 - Current best internal CV score: 0.6599999999999999

Generation 10 - Current best internal CV score: 0.6599999999999999

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=9, min_samples_leaf=15, min_samples_split=15)


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7

Generation 2 - Current best internal CV score: 0.7

Generation 3 - Current best internal CV score: 0.7

Generation 4 - Current best internal CV score: 0.7

Generation 5 - Current best internal CV score: 0.74

Generation 6 - Current best internal CV score: 0.74

Generation 7 - Current best internal CV score: 0.74

Generation 8 - Current best internal CV score: 0.74

Generation 9 - Current best internal CV score: 0.74

Generation 10 - Current best internal CV score: 0.74

Best pipeline: GradientBoostingClassifier(ZeroCount(input_matrix), learning_rate=0.1, max_depth=4, max_features=0.35000000000000003, min_samples_leaf=6, min_samples_split=10, n_estimators=100, subsample=0.35000000000000003)


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]

...

Optimization Progress:   0%|          | 0/200 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7

3.01 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(KNeighborsClassifier(input_matrix, n_neighbors=90, p=1, weights=uniform), bootstrap=False, criterion=gini, max_features=0.25, min_samples_leaf=4, min_samples_split=9, n_estimators=100)


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6533333333333333

Generation 2 - Current best internal CV score: 0.6533333333333333

Generation 3 - Current best internal CV score: 0.6799999999999999

Generation 4 - Current best internal CV score: 0.6799999999999999

Generation 5 - Current best internal CV score: 0.6799999999999999

Generation 6 - Current best internal CV score: 0.6799999999999999

Generation 7 - Current best internal CV score: 0.6799999999999999

Generation 8 - Current best internal CV score: 0.6799999999999999

3.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=5, max_features=0.6000000000000001, min_samples_leaf=13, min_samples_split=14, n_estimators=100, subsample=0.6000000000000001)


GridSearchCV(cv=3,
             estimator=TPOTClassifier(max_eval_time_mins=0.04, max_time_mins=3,
                                      random_state=101, verbosity=2),
             param_grid={'generations': [10, 20, 30],
                         'population_size': [50, 100, 200]},
             scoring='accuracy')

In [68]:
grid_search.score(X_test,y_test)

0.48

In [69]:
preds = grid_search.predict(X_test)
print(preds)

[1 1 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0]
